In [ ]:
%pip install panda plotly seaborn unidecode

In [ ]:
import pandas as pd

base_df = pd.read_csv(
    "./pagos-2018-2024-salud-metropolitano.csv",
    delimiter=",",
    dtype={
        "capitulo": "str",
        "adquisicion_de_activos_no_financieros": "int64",
        "bienes_y_servicios_de_consumo": "int64",
        "gastos_en_personal": "int64",
        "iniciativas_de_inversion": "int64",
        "otros_gastos_corrientes": "int64",
        "prestaciones_de_seguridad_social": "int64",
        "servicio_de_la_deuda": "int64",
        "transferencias_corrientes": "int64",
    },
).rename(columns={"periodo": "año"})
base_df["mes_año"] = (
    base_df["mes"].astype(str).apply(lambda n: n.zfill(2))
    + "-"
    + base_df["año"].astype(str)
)
base_df = base_df[
    [
        "año",
        "mes",
        "mes_año",
        "capitulo",
        "nombre_capitulo",
        "adquisicion_de_activos_no_financieros",
        "bienes_y_servicios_de_consumo",
        "gastos_en_personal",
        "iniciativas_de_inversion",
        "otros_gastos_corrientes",
        "prestaciones_de_seguridad_social",
        "servicio_de_la_deuda",
        "transferencias_corrientes",
    ]
]

print(base_df.info())
base_df

In [ ]:
def sum_gasto_by(columns, sum_slice, df):
    df = df.groupby(columns).sum().reset_index()
    df["total_gasto"] = df.iloc[:, sum_slice].sum(axis=1)
    # df = df.drop(columns=df.columns[slice(sum_slice.start, -1)])

    return df


ejecucion_mes_anio_df = sum_gasto_by(
    [
        "año",
        "mes",
        "capitulo",
        "nombre_capitulo",
    ],
    slice(5, None),
    base_df,
)

ejecucion_mes_anio_df

In [ ]:
import plotly.express as px

fig = px.bar(
    ejecucion_mes_anio_df, x="mes_año", y="total_gasto", color="nombre_capitulo"
)
fig.update_layout(
    # yaxis=dict(tickformat="$"),
    xaxis_title="Mes-Año",
    yaxis_title="Total Gasto",
    title="Ejecución Presupuestaria por Mes-Año de Servicios de Salud de la RM",
)
fig.show()

In [ ]:
ejecucion_anio_df = sum_gasto_by(
    [
        "año",
        "capitulo",
        "nombre_capitulo",
    ],
    slice(3, None),
    base_df.drop(columns=["mes_año"]),
)
ejecucion_anio_df

fig2 = px.bar(ejecucion_anio_df, x="año", y="total_gasto", color="nombre_capitulo")
fig2.update_layout(
    # yaxis=dict(tickformat="$"),
    xaxis_title="Año",
    yaxis_title="Total Gasto",
    title="Ejecución Presupuestaria por Año de Servicios de Salud de la RM",
)
fig2.show()

## Ingresos Salud Mental


In [ ]:
class Servicio:
    ORIENTE = 42
    CENTRAL = 43
    SUR = 44
    NORTE = 45
    OCCIDENTE = 46
    SURORIENTE = 47


def read_ingresos_salud_mental(path, servicio_id):
    df = pd.read_csv(
        path,
        delimiter=",",
        names=["año", "total", "glosa"],
        skiprows=1,
    )
    df["capitulo"] = str(servicio_id)
    df = df.pivot_table(
        index=["año", "capitulo"], columns="glosa", values="total", aggfunc="sum"
    ).reset_index()
    df.columns = df.columns.str.lower().str.replace(" ", "_")

    return df


salud_mental_df = pd.concat(
    [
        read_ingresos_salud_mental(
            "./datasets/ingresos-salud-mental/metropolitano-oriente.csv",
            Servicio.ORIENTE,
        ),
        read_ingresos_salud_mental(
            "./datasets/ingresos-salud-mental/metropolitano-central.csv",
            Servicio.CENTRAL,
        ),
        read_ingresos_salud_mental(
            "./datasets/ingresos-salud-mental/metropolitano-sur.csv",
            Servicio.SUR,
        ),
        read_ingresos_salud_mental(
            "./datasets/ingresos-salud-mental/metropolitano-norte.csv",
            Servicio.NORTE,
        ),
        read_ingresos_salud_mental(
            "./datasets/ingresos-salud-mental/metropolitano-occidente.csv",
            Servicio.OCCIDENTE,
        ),
        read_ingresos_salud_mental(
            "./datasets/ingresos-salud-mental/metropolitano-suroriente.csv",
            Servicio.SURORIENTE,
        ),
    ],
    axis=0,
    ignore_index=True,
)


salud_mental_df

In [ ]:
presupuesto_salud_mental_df = pd.merge(
    ejecucion_anio_df, salud_mental_df, on=["capitulo", "año"], how="left"
)

# TODO: imputar datos para 2018 y 2024 (?)
presupuesto_salud_mental_df["año"] = presupuesto_salud_mental_df["año"].astype(int)
presupuesto_salud_mental_df["capitulo"] = presupuesto_salud_mental_df[
    "capitulo"
].astype(int)
# joined_df = impute_data("total_gasto", "año", "avg", joined_df)
# presupuesto_salud_mental_df = (
#     presupuesto_salud_mental_df.groupby(["año", "capitulo", "nombre_capitulo"])
#     .apply(lambda group: group.interpolate(method="linear"))
#     .reset_index(drop=True)
# )
presupuesto_salud_mental_df

## CASEN


In [ ]:
raw_casen_df = pd.read_csv(
    "./datasets/casen/casen-2017-2022-metropolitana-fonasa.csv",
    delimiter=",",
)
raw_casen_df

In [ ]:
years = range(2017, 2025)
capitulos = range(42, 48)
nombres_capitulos = list(raw_casen_df[raw_casen_df["año"] == 2017]["nombre_capitulo"])

new_rows = pd.DataFrame(
    [
        (year, capitulo, nombre_capitulo)
        for year in years
        for capitulo, nombre_capitulo in zip(capitulos, nombres_capitulos)
    ],
    columns=["año", "capitulo", "nombre_capitulo"],
)

imputed_casen_df = (
    pd.merge(
        raw_casen_df, new_rows, on=["año", "capitulo", "nombre_capitulo"], how="outer"
    )
    .groupby(["capitulo", "nombre_capitulo"])
    .apply(lambda group: group.interpolate(method="linear"))
    .reset_index(drop=True)
)

casen_df = imputed_casen_df.sort_values(
    by=["año", "capitulo"], ascending=True
).reset_index(drop=True)

casen_df

## Egresos Hospitalarios


In [ ]:
from unidecode import unidecode

DIAGNOSTICO_MAPPING = {
    "Acute bronchitis": "Bronquitis aguda",
    "Encounter for attention to artificial openings": "Encuentro para atención a aperturas artificiales",
    "Endometriosis": "Endometriosis",
    "Gastric ulcer": "Úlcera gástrica",
    "Malignant neoplasm of bladder": "Neoplasia maligna de la vejiga",
    "Malignant neoplasm of colon": "Neoplasia maligna del colon",
    "Malignant neoplasm of stomach": "Neoplasia maligna del estómago",
    "Nontraumatic intracerebral hemorrhage": "Hemorragia intracerebral no traumática",
    "Other aneurysm": "Otros aneurismas",
    "Other diseases of anus and rectum": "Otras enfermedades del ano y recto",
    "Duodenal ulcer": "Úlcera duodenal",
    "Hemorrhoids and perianal venous thrombosis": "Hemorroides y trombosis venosa perianal",
    "Malignant neoplasm of brain": "Neoplasia maligna del cerebro",
    "Abnormalities of forces of labor": "Anomalías de las fuerzas del trabajo de parto",
    "Other congenital malformations of intestine": "Otras malformaciones congénitas del intestino",
    "Other cerebrovascular diseases": "Otras enfermedades cerebrovasculares",
    "Ventral hernia": "Hernia ventral",
}

raw_egresos_df = pd.read_csv(
    "./datasets/dias_cama_top_10_diag_por_servicio.csv",
    delimiter=",",
    dtype={"año": "int16", "capitulo": "int16"},
).drop(columns=["index", "nombre_capitulo_resumido"])

raw_egresos_df["diagnostico"] = raw_egresos_df["diagnostico"].apply(
    lambda diag: DIAGNOSTICO_MAPPING[diag]
)
raw_egresos_df = raw_egresos_df[raw_egresos_df["año"] >= 2018]

pivoted_egresos_df = raw_egresos_df.pivot_table(
    index=["año", "capitulo", "nombre_capitulo"],
    columns="diagnostico",
    values="dias_cama_promedio",
).reset_index()
pivoted_egresos_df.columns = pivoted_egresos_df.columns.str.lower().str.replace(
    " ", "_"
)
pivoted_egresos_df.columns = [
    unidecode(column) if column != "año" else "año"
    for column in pivoted_egresos_df.columns
]
pivoted_egresos_df

In [ ]:
joined_df = pd.merge(
    pd.merge(
        presupuesto_salud_mental_df,
        casen_df,
        on=["año", "capitulo", "nombre_capitulo"],
        how="left",
    ),
    pivoted_egresos_df,
    on=["año", "capitulo", "nombre_capitulo"],
    how="left",
).fillna(0)

joined_df

In [ ]:
suroriente_df = joined_df[joined_df["capitulo"] == 47]
suroriente_df

In [ ]:
joined_df.info()

In [ ]:
joined_df

In [ ]:
corr_matrix = joined_df[
    [
        "adquisicion_de_activos_no_financieros",
        "bienes_y_servicios_de_consumo",
        "gastos_en_personal",
        "iniciativas_de_inversion",
        "otros_gastos_corrientes",
        "total_problemas_conseguir_cita",
        "total_problemas_atencion",
        "total_problemas_pagar",
        "total_problemas_entrega_medicamento",
    ]
].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale="Plasma_r",
    aspect="auto",
    labels=dict(color="Correlación"),
)
fig.update_layout(height=700)
fig.show()

In [ ]:
# import seaborn as sns

# sns.set_theme()

# sns.pairplot(
#     joined_df,
#     x_vars=[
#         "año",
#         "capitulo",
#         "total_gasto",
#         "suicidio_ideacion",
#         "suicidio_intento",
#     ],
#     y_vars=[
#         "año",
#         "capitulo",
#         "total_gasto",
#         "suicidio_ideacion",
#         "suicidio_intento",
#     ],
#     hue="nombre_capitulo",
#     markers=["o", "s", "h", "v", "8", "D"],
#     #     diag_kind="hist",
#     #     # corner=True,
# )

In [ ]:
import plotly.express as px

SALUD_MENTAL_OPTIONS = {
    "x": "año",
    "color": "nombre_capitulo",
    "line_shape": "linear",
    "labels": {"nombre_capitulo": "Servicio"},
}

fig1 = px.line(
    joined_df,
    y="total_gasto",
    **SALUD_MENTAL_OPTIONS,
)

fig1.update_layout(
    # yaxis=dict(tickformat="$"),
    xaxis_title="Año",
    yaxis_title="Total Gasto",
    title="Ejecución Presupuestaria por Año de Servicios de Salud de la RM",
)

fig1.show()

In [ ]:
fig2 = px.line(
    joined_df,
    y="suicidio_ideacion",
    **SALUD_MENTAL_OPTIONS,
)

fig2.update_layout(
    xaxis_title="Año",
    yaxis_title="Cantidad",
    title="Ingresos a Programa de Salud Mental por Ideación de Suicidio por Año de la RM",
)

fig2.show()

In [ ]:
fig3 = px.line(
    joined_df,
    y="suicidio_intento",
    **SALUD_MENTAL_OPTIONS,
)

fig3.update_layout(
    xaxis_title="Año",
    yaxis_title="Cantidad",
    title="Ingresos a Programa de Salud Mental por Intento de Suicidio por Año de la RM",
)

fig3.show()